# 배연주 참고

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [5]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [6]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [7]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    
    lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.2)
    model.add(keras.layers.Dense(300, activation=lrelu))
    model.add(keras.layers.Dense(150, activation=lrelu))
    model.add(keras.layers.Dense(100, activation=lrelu))
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])
    return model

In [8]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 2s 5ms/step - loss: 0.3213 - accuracy: 0.8888
processing fold # 1
188/188 [==============================] - 1s 4ms/step - loss: 0.3032 - accuracy: 0.8970
processing fold # 2
188/188 [==============================] - 1s 3ms/step - loss: 0.3138 - accuracy: 0.8938
processing fold # 3
188/188 [==============================] - 1s 3ms/step - loss: 0.3465 - accuracy: 0.8852
processing fold # 4
188/188 [==============================] - 1s 4ms/step - loss: 0.3380 - accuracy: 0.8872
processing fold # 5
188/188 [==============================] - 1s 3ms/step - loss: 0.3180 - accuracy: 0.8918
processing fold # 6
188/188 [==============================] - 1s 3ms/step - loss: 0.3099 - accuracy: 0.8978
processing fold # 7
188/188 [==============================] - 1s 3ms/step - loss: 0.3358 - accuracy: 0.8882
processing fold # 8
188/188 [==============================] - 1s 3ms/step - loss: 0.3824 - accuracy: 0.8748
processing fold # 9

In [9]:
all_scores

[0.8888333439826965,
 0.8970000147819519,
 0.893833339214325,
 0.8851666450500488,
 0.8871666789054871,
 0.8918333053588867,
 0.8978333473205566,
 0.8881666660308838,
 0.874833345413208,
 0.8911666870117188]

In [10]:
np.mean(all_scores)

0.8895833373069764